In [1]:
import pandas as pd
import numpy as np
import functools as ft
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
COLORS = [
    '#00B0F0',
    '#FF0000'
]

In [3]:
dfs = []
for x in range(2004, 2024):
    y = pd.read_csv(f'data/{x}.csv', sep='\t').dropna(thresh=2).reset_index(drop=True)[['Country', f'{x-1}', f'{x}', f'{x+1}']].rename({
        f'{x-1}': f'{x-1}_in_{x}',
        f'{x}': f'{x}_in_{x}',
        f'{x+1}': f'{x+1}_in_{x}',
    }, axis=1)
    y[f'{x-1}_in_{x}'] = y[f'{x-1}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    y[f'{x}_in_{x}'] = y[f'{x}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    y[f'{x+1}_in_{x}'] = y[f'{x+1}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    dfs.append(y)
df = ft.reduce(lambda left, right: pd.merge(left, right, on='Country'), dfs)
df.dropna(inplace=True)
df.head()

,Country,2003_in_2004,2004_in_2004,2005_in_2004,2004_in_2005,2005_in_2005,2006_in_2005,2005_in_2006,2006_in_2006,2007_in_2006,...,2021_in_2020,2020_in_2021,2021_in_2021,2022_in_2021,2021_in_2022,2022_in_2022,2023_in_2022,2022_in_2023,2023_in_2023,2024_in_2023
0,Antigua and Barbuda,2.5,1.0,0.5,4.1,2.4,2.6,5.0,7.1,3.9,...,4.701,-20.031,0.965,7.019,5.270,6.021,5.629,8.459,5.592,5.417
1,Argentina,8.8,7.0,4.0,9.0,7.5,4.2,9.2,8.0,6.0,...,4.889,-9.905,7.500,2.465,10.398,4.037,2.024,4.956,-2.500,2.750
2,Barbados,2.2,3.0,2.5,4.4,3.1,3.3,3.9,4.2,4.9,...,7.400,-18.000,3.300,8.500,0.700,10.500,5.000,9.800,4.500,3.900
3,Belize,9.4,3.0,3.3,4.6,2.2,2.7,3.5,5.3,2.6,...,8.000,-14.038,8.500,5.400,16.342,3.502,1.998,12.726,4.007,2.998
4,Bolivia,2.5,3.8,4.5,3.6,3.9,2.5,4.1,4.1,3.9,...,5.600,-8.829,4.950,4.000,6.106,3.800,3.200,3.478,1.800,1.800


In [4]:
df[df['Country'] == 'Guyana'][['2020_in_2021', '2021_in_2022', '2022_in_2023']]

,2020_in_2021,2021_in_2022,2022_in_2023
15,43.48,23.774,62.288


In [5]:
# wtf?!
df = df[df['Country'] != 'Guyana']

In [6]:
for x in range(2004, 2023):
    df[f'expected_growth_{x}_t'] = df[f'{x}_in_{x}'].astype(float)
    df[f'expected_growth_{x}_t1'] = df[f'{x+1}_in_{x}'].astype(float)
    df[f'actual_growth_{x}'] = df[f'{x}_in_{x+1}'].astype(float)
df[['expected_growth_2008_t', 'expected_growth_2008_t1', 'actual_growth_2008']].head()

,expected_growth_2008_t,expected_growth_2008_t1,actual_growth_2008
0,2.092,4.009,2.800
1,6.500,3.600,6.758
2,1.700,1.000,0.180
3,4.000,2.500,3.771
4,5.900,5.000,6.100


In [7]:
base = {}
for x in range(2005, 2023):
    base[f'year_{x}'] = 0
for c in df['Country'].unique():
    base[f'country_{c}'] = 0

table = []
for x in range(2005, 2023):
    for c in df['Country'].unique():
        table.append({
            **base,
            'country': c,
            f'year_{x}': 1,
            f'country_{c}': 1,
            'expected_growth_t': df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
            'expected_growth_t1': df[f'expected_growth_{x}_t1'][df['Country'] == c].iloc[0] - df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
            'expected_growth_error': df[f'actual_growth_{x}'][df['Country'] == c].iloc[0] - df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
        })
X = pd.DataFrame(table).drop(['country_Mexico', 'year_2005'], axis=1)
groups = X['country']
X.drop('country', axis=1, inplace=True)
X.head()

,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,...,country_St. Kitts and Nevis,country_St. Lucia,country_St. Vincent and the Grenadines,country_Suriname,country_Trinidad and Tobago,country_Uruguay,country_Venezuela,expected_growth_t,expected_growth_t1,expected_growth_error
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.4,0.2,2.6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.5,-3.3,1.7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.1,0.2,0.8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.2,0.5,1.3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.9,-1.4,0.2


In [8]:
y = X['expected_growth_error']
model = sm.OLS(y, sm.add_constant(X.drop(['expected_growth_error'], axis=1)))
fitted_model = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
fitted_model.summary()

/home/seppo/.local/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 47, but rank is 19
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     expected_growth_error   R-squared:                       0.233
Model:                               OLS   Adj. R-squared:                  0.157
Method:                    Least Squares   F-statistic:                     36.71
Date:                   Thu, 20 Feb 2025   Prob (F-statistic):           6.14e-15
Time:                           22:18:25   Log-Likelihood:                -1031.8
No. Observations:                    522   AIC:                             2160.
Df Residuals:                        474   BIC:                             2364.
Df Model:                             47                                         
Covariance Type:                 cluster                                         
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      0.1015      0.367      0.277      0.782      -0.618       0.821
year_2006                                  0.0604      0.356      0.170      0.865      -0.637       0.758
year_2007                                 -0.5733      0.358     -1.603      0.109      -1.274       0.128
year_2008                                 -1.2581      0.302     -4.160      0.000      -1.851      -0.665
year_2009                                 -0.8390      0.442     -1.900      0.057      -1.705       0.027
year_2010                                 -0.3638      0.405     -0.897      0.370      -1.158       0.431
year_2011                                 -1.2685      0.474     -2.674      0.007      -2.198      -0.339
year_2012                                 -0.9128      0.318     -2.868      0.004      -1.537      -0.289
year_2013                                 -0.5332      0.295     -1.808      0.071      -1.111       0.045
year_2014                                 -0.4873      0.293     -1.665      0.096      -1.061       0.086
year_2015                                 -0.7609      0.373     -2.042      0.041      -1.491      -0.031
year_2016                                 -1.0325      0.399     -2.590      0.010      -1.814      -0.251
year_2017                                 -1.0389      0.512     -2.028      0.043      -2.043      -0.035
year_2018                                 -0.2327      0.467     -0.498      0.619      -1.149       0.683
year_2019                                 -1.2619      0.367     -3.434      0.001      -1.982      -0.542
year_2020                                 -2.0791      1.254     -1.658      0.097      -4.537       0.379
year_2021                                  1.0899      0.671      1.623      0.105      -0.226       2.406
year_2022                                  0.0883      0.534      0.165      0.869      -0.959       1.135
country_Antigua and Barbuda                0.6478      0.070      9.244      0.000       0.510       0.785
country_Argentina                          1.1365      0.097     11.690      0.000       0.946       1.327
country_Barbados                          -0.8498      0.035    -24.336      0.000      -0.918      -0.781
country_Belize                             0.6134      0.012     50.263      0.000       0.589       0.637
country_Bolivia                           -0.0822      0.090     -0.915      0.360      -0.258       0.094
country_Brazil                            -0.0439      0.052     -0.847      0.397      -0.145       0.058
country_Chile                             -0.3300      0.076     -4.327      0.000      -0.480      -0.181
country_Colombia                           0.3411     

In [9]:
y.mean(), y.std()

(0.26620881226053644, 1.9962361936801627)